In [1]:
# numerical computation
import numpy as np
import itertools
from random import randint


# import sklearn

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import hamming_loss
from sklearn.model_selection import GridSearchCV

# dataframe management
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=UserWarning)


In [2]:
def opened (path=''):
    
    X_training=[]
    X_testing=[]
    y_training=[]
    y_testing=[]
           
    for j in range(0, 50):
        X_training.append(pd.read_csv('test_train_dataset{}{}_X_train.csv'.format(path,j)))
        X_testing.append(pd.read_csv('test_train_dataset{}{}_X_test.csv'.format(path, j)))
        y_training.append(pd.read_csv('test_train_dataset{}{}_y_train.csv'.format(path, j)))
        y_testing.append(pd.read_csv('test_train_dataset{}{}_y_test.csv'.format(path, j)))
        
    return X_training, X_testing, y_training, y_testing

In [3]:
def frequency (valor):
    max = 0
    res = list(valor)[0] 
    for i in list(valor): 
        freq = list(valor).count(i) 
        if freq > max: 
            max = freq 
            res = i 
    valor = res
    return valor

In [4]:
def maximun (df, name):
    maximun = df.sort_values(by='accuracy_validation',ascending=False).head(n=1)
    best = list(maximun[name])[0]
    return best

In [5]:
def parameters(i):
    max_layers = 60#204
    min_layers = 10#5
    switcher={
            'activation':[ 'tanh', 'relu'],
            'solver': ['sgd'], #['sgd', 'adam'],
            'hidden_layer_sizes':[(i,) for i in range(min_layers,max_layers, 2)]
         }
    return switcher.get(i,"Invalid parameters")

In [6]:
def Layersint(Layers):
    Layers = tuple(Layers)
    maxi=len(Layers)
    cal=0
    res=[]
    for i in range(0, maxi):
        if Layers[i]=="(" or Layers[i]=="," or Layers[i]==")":
            res += str(Layers[i])
        else:
            if maxi==6:
                if i==1:
                    cal = int(Layers[i])*100
                elif i==2:
                    cal += int(Layers[i])*10
                else:
                    cal +=int(Layers[i])
            elif maxi==5:
                if i==1:
                    cal = int(Layers[i])*10
                else:
                    cal += int(Layers[i])
            else:
                cal = int(Layers[i])

    return cal

#  Aplicación del algoritmo MLP

In [7]:
def hyper_MLP(path, features, name, param, multiclass=False):
    
    x_train, x_test, y_train, y_test = opened(path=path)
    print('Terminada la apertura de BBDD')
    
    param_grid = [
        {
            param: parameters(param)
        }
       ]
    MLP_evaluate=[]
    MLP_acc_model=[]
    MLP_std=[]

    best_param=[]
    
    if param == 'hidden_layer_sizes':
        rest = ['activation', 'solver']
        resto=[]
        for r in rest:
            resto.append(pd.read_csv('results/MLP/MLP_hyper_{}_{}.csv'.format(r, name)))

        Activation = (maximun(resto[0], rest[0]))
        print(Activation)
        Solver = (maximun(resto[1], rest[1]))
        print(Solver)
        
        model = MLPClassifier(max_iter=400, learning_rate_init=0.2, 
                              learning_rate='invscaling', alpha = 1.0, 
                              solver=Solver, activation=Activation) 
    else:
        model = MLPClassifier(max_iter=400, learning_rate_init=0.2,
                              learning_rate='invscaling', alpha = 1.0)
        
    for j in range(0, 50):
        
        droping=pd.concat([x_train[j][features], y_train[j]], axis=1,sort=False)
        droping=droping.drop_duplicates(subset=features, keep=False)
        xtrain= droping[features]
        if multiclass==True:
            ytrain=droping['CRG']
        else:
            ytrain=droping[['HP', 'Diabetes', 'Otros']]
                
        print('Particion: ', j)
    #Normalizamos los datos de test y de train
        ss=StandardScaler()
        ss.fit(xtrain)
        ss_train=ss.transform(xtrain)
        #Buscamos los mejores parametros para esa división normalizada    
        clf = GridSearchCV(model, param_grid,
                           cv=KFold(n_splits=5), scoring='accuracy', n_jobs=-1)
        if multiclass==True:
            y_training = ytrain.values.ravel()
        else:
            y_training = ytrain

        clf.fit(ss_train,y_training)

        best_index_Acc = np.nonzero(clf.cv_results_['rank_test_score'] == 1)[0][0]
        best_param.append(clf.best_params_[param])
        MLP_acc_model.append(clf.cv_results_['mean_test_score'][best_index_Acc])
        MLP_std.append(clf.cv_results_['std_test_score'][best_index_Acc])

        MLP_evaluate.append([best_param[j],  round(MLP_acc_model[j],3),round(MLP_std[j],3)])

    labels_comp = [param , 'accuracy_validation', 'std']

    comparacion=pd.DataFrame(data=MLP_evaluate, columns = labels_comp)
    comparacion.to_csv('results/MLP/MLP_hyper_{}_{}.csv'.format(param, name), index=False)

In [8]:
def predict_MLP(path, features, name, multiclass=False):
    
    x_train, x_test, y_train, y_test = opened(path=path)
    print('Terminada la apertura de BBDD')
    
    param= ['activation', 'solver', 'hidden_layer_sizes']
    comparacion=[]
    for p in param:
        comparacion.append(pd.read_csv('results/MLP/MLP_hyper_{}_{}.csv'.format(p, name)))
    
    Activation = (maximun(comparacion[0], param[0]))
    print(Activation)
    Solver = (maximun(comparacion[1], param[1]))
    print(Solver)
    Layer = Layersint(maximun(comparacion[2], param[2]))
    print(Layer)
    print('Con los parámetros óptimos procedemos a clasificar.')
    
    accuracy=[]
    hamming_losse=[]
    precision_macro=[]
    precision_micro=[]
    recall_macro=[]
    recall_micro=[]
    f1_scores_macro=[]
    f1_scores_micro=[]
    
    average_accuracy=[]
    average_precision=[]
    average_recall=[]
    f1_scores=[]
    
    for i in range(0,50):
        droping_train=pd.concat([x_train[i][features], y_train[i]], axis=1,sort=False)
        droping_train=droping_train.drop_duplicates(subset=features, keep=False)
        xtrain= droping_train[features]
        if multiclass==True:
            ytrain=droping_train['CRG']
        else:
            ytrain=droping_train[['HP', 'Diabetes', 'Otros']]

        droping_test=pd.concat([x_test[i][features], y_test[i]], axis=1,sort=False)
        droping=droping_test.drop_duplicates(subset=features, keep=False)
        xtest= droping_test[features]
        if multiclass==True:
            ytest=droping_test['CRG']
        else:
            ytest=droping_test[['HP', 'Diabetes', 'Otros']]
                
        ss=StandardScaler()
        ss.fit(xtrain)
        ss_train=ss.transform(xtrain)
        ss_test=ss.transform(xtest)

        clf= MLPClassifier(max_iter=400, learning_rate_init=0.2, activation=Activation, 
                           solver=Solver, hidden_layer_sizes= (Layer,), alpha = 1.0, 
                           learning_rate='invscaling')
        
        if multiclass==True:
            y_training = ytrain.values.ravel()
        else:
            y_training = ytrain
               
        clf.fit(ss_train,y_training)

    #Predecimos el algoritmo con el mejor K
        y_true, y_pred = ytest, clf.predict(ss_test)
        
        if multiclass==False:
            accuracy.append(accuracy_score(y_true, y_pred))
            hamming_losse.append(hamming_loss(y_true, y_pred))
            precision_macro.append(precision_score(y_true,y_pred, average='macro'))
            precision_micro.append(precision_score(y_true,y_pred, average='micro'))
            recall_macro.append(recall_score(y_true, y_pred, average='macro'))
            recall_micro.append(recall_score(y_true, y_pred, average='micro'))
            f1_scores_macro.append(f1_score(y_true, y_pred, average='macro'))
            f1_scores_micro.append(f1_score(y_true, y_pred, average='micro'))
        else:
            cm = confusion_matrix(y_true,y_pred)
            TP = np.diag(cm)
            FP = np.sum(cm, axis=0) - TP
            FN = np.sum(cm, axis=1) - TP
            #num_classes = len(TP)
            #TN = []
            #for i in range(num_classes):
            #    temp = np.delete(cm, i, 0)    # delete ith row
            #    temp = np.delete(temp, i, 1)  # delete ith column
            #    TN.append(sum(sum(temp)))

            precision=TP / (TP + FP)
            recall=TP / (TP + FN)

            average_precision.append(np.mean(TP / (TP + FP)))
            average_recall.append(np.mean(TP / (TP + FN)))
            f1_scores.append(np.mean(2*(precision*recall)/(precision+recall)))
            average_accuracy.append(accuracy_score(y_true, y_pred))
        
    predict=pd.DataFrame()
    if multiclass==False:
        predict['accuracy']=accuracy
        predict['hamming_loss'] = hamming_losse
        predict['precision_macro']=precision_macro
        predict['precision_micro']=precision_micro
        predict['recall_macro']=recall_macro
        predict['recall_micro']=recall_micro
        predict['f1_macro']=f1_scores_macro
        predict['f1_micro']=f1_scores_micro
    else:
        predict['accuracy']=average_accuracy
        predict['precision']=average_precision
        predict['recall']=average_recall
        predict['f1']=f1_scores
    predict.to_csv('results/MLP/MLP_predict_{}.csv'.format(name), index=False)

### Selección de Caracteriticas: Frecuencia

In [9]:
import os.path as path
param= ['activation', 'solver', 'hidden_layer_sizes']

In [10]:
names = ['ocurrencia_all', 'ocurrencia_ill', 'presencia_all', 'presencia_ill'] 
features_freq = []
for n in names:
    with open("feature_selection/freq_{}.txt".format(n), "r") as file:
        features_freq.append(eval(file.readline()))

In [11]:
paths_CLASS = ['/class/O_WC_A_','/class/O_WC_WO_' , '/class/P_WC_A_', '/class/P_WC_WO_']
names_CLASS_fr=['freq_all_class_O', 'freq_ill_class_O', 'freq_all_class_P', 'freq_ill_class_P']

In [12]:
for p, n, f in zip(paths_CLASS, names_CLASS_fr, features_freq):
    for m in param:        
        if path.exists('results/MLP/MLP_hyper_{}_{}.csv'.format(m,n)): 
            print('Ya existe el hyperparametro:', n, m)
        else:
            hyper_MLP(p, f, n, m, True)
            print()
            print('--------------------------------------------------------')
            print()

    if path.exists('results/MLP/MLP_predict_{}.csv'.format(n)): 
        print('Ya existe los resultados:', n)
    else:
        predict_MLP(p, f, n, True)
        print()
        print('--------------------------------------------------------')
        print()

Ya existe el hyperparametro: freq_all_class_O activation
Ya existe el hyperparametro: freq_all_class_O solver
Ya existe el hyperparametro: freq_all_class_O hidden_layer_sizes
Ya existe los resultados: freq_all_class_O
Ya existe el hyperparametro: freq_ill_class_O activation
Ya existe el hyperparametro: freq_ill_class_O solver
Ya existe el hyperparametro: freq_ill_class_O hidden_layer_sizes
Ya existe los resultados: freq_ill_class_O
Ya existe el hyperparametro: freq_all_class_P activation
Ya existe el hyperparametro: freq_all_class_P solver
Ya existe el hyperparametro: freq_all_class_P hidden_layer_sizes
Ya existe los resultados: freq_all_class_P
Ya existe el hyperparametro: freq_ill_class_P activation
Ya existe el hyperparametro: freq_ill_class_P solver
Ya existe el hyperparametro: freq_ill_class_P hidden_layer_sizes
Ya existe los resultados: freq_ill_class_P


In [13]:
paths_LABEL = ['/label/O_WL_A_','/label/O_WL_WO_' , '/label/P_WL_A_', '/label/P_WL_WO_']
names_LABEL_fr=['freq_all_label_O', 'freq_ill_label_O', 'freq_all_label_P', 'freq_ill_label_P']

In [14]:
for p, n, f in zip(paths_LABEL, names_LABEL_fr, features_freq):
    for m in param:        
        if path.exists('results/MLP/MLP_hyper_{}_{}.csv'.format(m,n)): 
            print('Ya existe el hyperparametro:', n, m)
        else:
            hyper_MLP(p, f, n, m)
            print()
            print('--------------------------------------------------------')
            print()
    
    if path.exists('results/MLP/MLP_predict_{}.csv'.format(n)): 
        print('Ya existe los resultados:', n)
    else:
        predict_MLP(p, f, n)
        print()
        print('--------------------------------------------------------')
        print()

Ya existe el hyperparametro: freq_all_label_O activation
Ya existe el hyperparametro: freq_all_label_O solver
Ya existe el hyperparametro: freq_all_label_O hidden_layer_sizes
Ya existe los resultados: freq_all_label_O
Ya existe el hyperparametro: freq_ill_label_O activation
Ya existe el hyperparametro: freq_ill_label_O solver
Ya existe el hyperparametro: freq_ill_label_O hidden_layer_sizes
Ya existe los resultados: freq_ill_label_O
Ya existe el hyperparametro: freq_all_label_P activation
Ya existe el hyperparametro: freq_all_label_P solver
Ya existe el hyperparametro: freq_all_label_P hidden_layer_sizes
Ya existe los resultados: freq_all_label_P
Ya existe el hyperparametro: freq_ill_label_P activation
Ya existe el hyperparametro: freq_ill_label_P solver
Ya existe el hyperparametro: freq_ill_label_P hidden_layer_sizes
Ya existe los resultados: freq_ill_label_P


### Selección de Caracteriticas: Random Forest

In [15]:
names=['label_o_all','label_o_ill', 'label_p_all', 'label_p_ill']
features_rf_label = []
for n in names:
    with open("feature_selection/rf_{}.txt".format(n), "r") as file:
        features_rf_label.append(eval(file.readline()))

In [16]:
path_label= ['/label/O_WL_A_', '/label/O_WL_WO_', '/label/P_WL_A_', '/label/P_WL_WO_']
names_label_rf=['rf_all_label_O','rf_ill_label_O', 'rf_all_label_P', 'rf_ill_label_P']

In [17]:
for p, n, f in zip(path_label, names_label_rf, features_rf_label):
    for m in param:        
        if path.exists('results/MLP/MLP_hyper_{}_{}.csv'.format(m,n)): 
            print('Ya existe el hyperparametro:', n, m)
        else:
            hyper_MLP(p, f, n, m)
            print()
            print('--------------------------------------------------------')
            print()
    
    if path.exists('results/MLP/MLP_predict_{}.csv'.format(n)): 
        print('Ya existe los resultados:', n)
    else:
        predict_MLP(p, f, n)
        print()
        print('--------------------------------------------------------')
        print()

Ya existe el hyperparametro: rf_all_label_O activation
Ya existe el hyperparametro: rf_all_label_O solver
Ya existe el hyperparametro: rf_all_label_O hidden_layer_sizes
Ya existe los resultados: rf_all_label_O
Ya existe el hyperparametro: rf_ill_label_O activation
Ya existe el hyperparametro: rf_ill_label_O solver
Ya existe el hyperparametro: rf_ill_label_O hidden_layer_sizes
Ya existe los resultados: rf_ill_label_O
Ya existe el hyperparametro: rf_all_label_P activation
Ya existe el hyperparametro: rf_all_label_P solver
Ya existe el hyperparametro: rf_all_label_P hidden_layer_sizes
Ya existe los resultados: rf_all_label_P
Ya existe el hyperparametro: rf_ill_label_P activation
Ya existe el hyperparametro: rf_ill_label_P solver
Ya existe el hyperparametro: rf_ill_label_P hidden_layer_sizes
Ya existe los resultados: rf_ill_label_P


In [18]:
names=['class_o_all','class_o_ill', 'class_p_all', 'class_p_ill']
features_rf_class = []
for n in names:
    with open("feature_selection/rf_{}.txt".format(n), "r") as file:
        features_rf_class.append(eval(file.readline()))

In [19]:
path_class= ['/class/O_WC_A_', '/class/O_WC_WO_', '/class/P_WC_A_', '/class/P_WC_WO_']
name_class_rf=['rf_all_class_O','rf_ill_class_O', 'rf_all_class_P', 'rf_ill_class_P']

In [20]:
for p, n, f in zip(path_class, name_class_rf, features_rf_class):
    for m in param:        
        if path.exists('results/MLP/MLP_hyper_{}_{}.csv'.format(m,n)): 
            print('Ya existe el hyperparametro:', n, m)
        else:
            hyper_MLP(p, f, n, m,True)
            print()
            print('--------------------------------------------------------')
            print()
    
    if path.exists('results/MLP/MLP_predict_{}.csv'.format(n)): 
        print('Ya existe los resultados:', n)
    else:
        predict_MLP(p, f, n, True)
        print()
        print('--------------------------------------------------------')
        print()

Ya existe el hyperparametro: rf_all_class_O activation
Ya existe el hyperparametro: rf_all_class_O solver
Ya existe el hyperparametro: rf_all_class_O hidden_layer_sizes
Ya existe los resultados: rf_all_class_O
Ya existe el hyperparametro: rf_ill_class_O activation
Ya existe el hyperparametro: rf_ill_class_O solver
Ya existe el hyperparametro: rf_ill_class_O hidden_layer_sizes
Ya existe los resultados: rf_ill_class_O
Ya existe el hyperparametro: rf_all_class_P activation
Ya existe el hyperparametro: rf_all_class_P solver
Ya existe el hyperparametro: rf_all_class_P hidden_layer_sizes
Ya existe los resultados: rf_all_class_P
Ya existe el hyperparametro: rf_ill_class_P activation
Ya existe el hyperparametro: rf_ill_class_P solver
Ya existe el hyperparametro: rf_ill_class_P hidden_layer_sizes
Ya existe los resultados: rf_ill_class_P


### Selección de caracteristicas: F Fisher

In [21]:
names=['label_o_all','label_o_ill', 'label_p_all', 'label_p_ill']
features_fc_label = []
for n in names:
    with open("feature_selection/fc_{}.txt".format(n), "r") as file:
        features_fc_label.append(eval(file.readline()))

In [22]:
path_label= ['/label/O_WL_A_', '/label/O_WL_WO_', '/label/P_WL_A_', '/label/P_WL_WO_']
names_label_fc=['fc_all_label_O','fc_ill_label_O', 'fc_all_label_P', 'fc_ill_label_P']

In [23]:
for p, n, f in zip(path_label, names_label_fc, features_fc_label):
    for m in param:        
        if path.exists('results/MLP/MLP_hyper_{}_{}.csv'.format(m,n)): 
            print('Ya existe el hyperparametro:', n, m)
        else:
            hyper_MLP(p, f, n, m)
            print()
            print('--------------------------------------------------------')
            print()
    
    if path.exists('results/MLP/MLP_predict_{}.csv'.format(n)): 
        print('Ya existe los resultados:', n)
    else:
        predict_MLP(p, f, n)
        print()
        print('--------------------------------------------------------')
        print()

Ya existe el hyperparametro: fc_all_label_O activation
Ya existe el hyperparametro: fc_all_label_O solver
Ya existe el hyperparametro: fc_all_label_O hidden_layer_sizes
Ya existe los resultados: fc_all_label_O
Ya existe el hyperparametro: fc_ill_label_O activation
Ya existe el hyperparametro: fc_ill_label_O solver
Ya existe el hyperparametro: fc_ill_label_O hidden_layer_sizes
Ya existe los resultados: fc_ill_label_O
Ya existe el hyperparametro: fc_all_label_P activation
Ya existe el hyperparametro: fc_all_label_P solver
Ya existe el hyperparametro: fc_all_label_P hidden_layer_sizes
Ya existe los resultados: fc_all_label_P
Ya existe el hyperparametro: fc_ill_label_P activation
Ya existe el hyperparametro: fc_ill_label_P solver
Ya existe el hyperparametro: fc_ill_label_P hidden_layer_sizes
Ya existe los resultados: fc_ill_label_P


In [24]:
names=['class_o_all','class_o_ill', 'class_p_all', 'class_p_ill']
features_fc_class = []
for n in names:
    with open("feature_selection/rf_{}.txt".format(n), "r") as file:
        features_fc_class.append(eval(file.readline()))

In [25]:
path_class= ['/class/O_WC_A_', '/class/O_WC_WO_', '/class/P_WC_A_', '/class/P_WC_WO_']
name_class_fc=['fc_all_class_O','fc_ill_class_O', 'fc_all_class_P', 'fc_ill_class_P']

In [26]:
for p, n, f in zip(path_class, name_class_fc, features_fc_class):
    for m in param:        
        if path.exists('results/MLP/MLP_hyper_{}_{}.csv'.format(m,n)): 
            print('Ya existe el hyperparametro:', n, m)
        else:
            hyper_MLP(p, f, n, m,True)
            print()
            print('--------------------------------------------------------')
            print()
    
    if path.exists('results/MLP/MLP_predict_{}.csv'.format(n)): 
        print('Ya existe los resultados:', n)
    else:
        predict_MLP(p, f, n, True)
        print()
        print('--------------------------------------------------------')
        print()

Ya existe el hyperparametro: fc_all_class_O activation
Ya existe el hyperparametro: fc_all_class_O solver
Ya existe el hyperparametro: fc_all_class_O hidden_layer_sizes
Ya existe los resultados: fc_all_class_O
Ya existe el hyperparametro: fc_ill_class_O activation
Ya existe el hyperparametro: fc_ill_class_O solver
Ya existe el hyperparametro: fc_ill_class_O hidden_layer_sizes
Ya existe los resultados: fc_ill_class_O
Ya existe el hyperparametro: fc_all_class_P activation
Ya existe el hyperparametro: fc_all_class_P solver
Ya existe el hyperparametro: fc_all_class_P hidden_layer_sizes
Ya existe los resultados: fc_all_class_P
Ya existe el hyperparametro: fc_ill_class_P activation
Ya existe el hyperparametro: fc_ill_class_P solver
Ya existe el hyperparametro: fc_ill_class_P hidden_layer_sizes
Ya existe los resultados: fc_ill_class_P


### Resultados:

In [27]:
def resultados_etiquetas(names):
    for n in  names:
        print(n)
        print()
        hyper=[]
        for p in param:
            hyper.append(pd.read_csv('results/MLP/MLP_hyper_{}_{}.csv'.format(p, n)))
        Activation = (maximun(hyper[0], param[0]))
        print(Activation)
        Solver = (maximun(hyper[1], param[1]))
        print(Solver)
        Layer = Layersint(maximun(hyper[2], param[2]))
        print(Layer)

        predict = pd.read_csv('results/MLP/MLP_predict_{}.csv'.format(n))

        print('Tasa de acierto:', round(np.mean(predict['accuracy']), 3), '+/-', round(np.std(predict['accuracy']), 3))
        print('Tasa de Hamming Loss:', round(np.mean(predict['hamming_loss']), 3), '+/-', round(np.std(predict['hamming_loss']), 3))
        print('Tasa de precision(macro)', round(np.mean(predict['precision_macro']), 3), '+/-', round(np.std(predict['precision_macro']), 3))
        print('Tasa de precision(micro)', round(np.mean(predict['precision_micro']), 3), '+/-', round(np.std(predict['precision_micro']), 3))
        print('Tasa de exactitud(macro):', round(np.mean(predict['recall_macro']), 3),  '+/-', round(np.std(predict['recall_macro']), 3))
        print('Tasa de exactitud(micro):', round(np.mean(predict['recall_micro']), 3),  '+/-', round(np.std(predict['recall_micro']), 3))
        print('Tasa F1-Score(macro)', round(np.mean(predict['f1_macro']), 3) , '+/-', round(np.std(predict['f1_macro']),3))
        print('Tasa F1-Score(micro)', round(np.mean(predict['f1_micro']), 3) , '+/-', round(np.std(predict['f1_micro']),3))
        print('---------------------------------------------------------------')

In [28]:
resultados_etiquetas(names_LABEL_fr)

freq_all_label_O

relu
sgd
185
Tasa de acierto: 0.799 +/- 0.016
Tasa de Hamming Loss: 0.074 +/- 0.007
Tasa de precision(macro) 0.904 +/- 0.015
Tasa de precision(micro) 0.942 +/- 0.008
Tasa de exactitud(macro): 0.838 +/- 0.016
Tasa de exactitud(micro): 0.896 +/- 0.011
Tasa F1-Score(macro) 0.868 +/- 0.013
Tasa F1-Score(micro) 0.919 +/- 0.007
---------------------------------------------------------------
freq_ill_label_O

relu
adam
180
Tasa de acierto: 0.753 +/- 0.023
Tasa de Hamming Loss: 0.093 +/- 0.009
Tasa de precision(macro) 0.891 +/- 0.021
Tasa de precision(micro) 0.927 +/- 0.015
Tasa de exactitud(macro): 0.85 +/- 0.023
Tasa de exactitud(micro): 0.913 +/- 0.012
Tasa F1-Score(macro) 0.867 +/- 0.015
Tasa F1-Score(micro) 0.92 +/- 0.007
---------------------------------------------------------------
freq_all_label_P

relu
sgd
155
Tasa de acierto: 0.844 +/- 0.014
Tasa de Hamming Loss: 0.057 +/- 0.005
Tasa de precision(macro) 0.915 +/- 0.012
Tasa de precision(micro) 0.95 +/- 0.007
Tasa d

In [29]:
resultados_etiquetas(names_label_fc)

fc_all_label_O

relu
sgd
180
Tasa de acierto: 0.842 +/- 0.015
Tasa de Hamming Loss: 0.058 +/- 0.006
Tasa de precision(macro) 0.941 +/- 0.011
Tasa de precision(micro) 0.957 +/- 0.008
Tasa de exactitud(macro): 0.886 +/- 0.015
Tasa de exactitud(micro): 0.917 +/- 0.011
Tasa F1-Score(macro) 0.912 +/- 0.011
Tasa F1-Score(micro) 0.937 +/- 0.007
---------------------------------------------------------------
fc_ill_label_O

relu
adam
120
Tasa de acierto: 0.823 +/- 0.025
Tasa de Hamming Loss: 0.067 +/- 0.01
Tasa de precision(macro) 0.937 +/- 0.017
Tasa de precision(micro) 0.949 +/- 0.015
Tasa de exactitud(macro): 0.904 +/- 0.015
Tasa de exactitud(micro): 0.936 +/- 0.01
Tasa F1-Score(macro) 0.919 +/- 0.01
Tasa F1-Score(micro) 0.943 +/- 0.008
---------------------------------------------------------------
fc_all_label_P

relu
sgd
155
Tasa de acierto: 0.903 +/- 0.011
Tasa de Hamming Loss: 0.035 +/- 0.004
Tasa de precision(macro) 0.955 +/- 0.012
Tasa de precision(micro) 0.968 +/- 0.007
Tasa de exac

In [30]:
resultados_etiquetas(names_label_rf)

rf_all_label_O

relu
sgd
135
Tasa de acierto: 0.815 +/- 0.018
Tasa de Hamming Loss: 0.069 +/- 0.007
Tasa de precision(macro) 0.919 +/- 0.013
Tasa de precision(micro) 0.946 +/- 0.008
Tasa de exactitud(macro): 0.861 +/- 0.014
Tasa de exactitud(micro): 0.905 +/- 0.01
Tasa F1-Score(macro) 0.888 +/- 0.011
Tasa F1-Score(micro) 0.925 +/- 0.008
---------------------------------------------------------------
rf_ill_label_O

relu
adam
135
Tasa de acierto: 0.786 +/- 0.023
Tasa de Hamming Loss: 0.081 +/- 0.009
Tasa de precision(macro) 0.918 +/- 0.018
Tasa de precision(micro) 0.939 +/- 0.013
Tasa de exactitud(macro): 0.874 +/- 0.022
Tasa de exactitud(micro): 0.922 +/- 0.014
Tasa F1-Score(macro) 0.893 +/- 0.014
Tasa F1-Score(micro) 0.93 +/- 0.008
---------------------------------------------------------------
rf_all_label_P

relu
sgd
170
Tasa de acierto: 0.877 +/- 0.015
Tasa de Hamming Loss: 0.044 +/- 0.005
Tasa de precision(macro) 0.935 +/- 0.014
Tasa de precision(micro) 0.96 +/- 0.008
Tasa de exac

In [31]:
def resultados_clases(names):
    for n in  names:
        print(n)
        print()
        hyper=[]
        for p in param:
            hyper.append(pd.read_csv('results/MLP/MLP_hyper_{}_{}.csv'.format(p, n)))
        Activation = (maximun(hyper[0], param[0]))
        print(Activation)
        Solver = (maximun(hyper[1], param[1]))
        print(Solver)
        Layer = Layersint(maximun(hyper[2], param[2]))
        print(Layer)

        predict = pd.read_csv('results/MLP/MLP_predict_{}.csv'.format(n))

        print('Tasa de acierto:', round(np.mean(predict['accuracy']), 3), '+/-', round(np.std(predict['accuracy']), 3))
        print('Tasa de precision', round(np.mean(predict['precision']), 3), '+/-', round(np.std(predict['precision']), 3))
        print('Tasa de exactitud:', round(np.mean(predict['recall']), 3),  '+/-', round(np.std(predict['recall']), 3))
        print('Tasa F1-Score', round(np.mean(predict['f1']), 3) , '+/-', round(np.std(predict['f1']),3))
        print('---------------------------------------------------------------')

In [32]:
resultados_clases(names_CLASS_fr)

freq_all_class_O

logistic
sgd
150
Tasa de acierto: 0.746 +/- 0.045
Tasa de precision 0.752 +/- 0.018
Tasa de exactitud: 0.746 +/- 0.045
Tasa F1-Score 0.751 +/- 0.018
---------------------------------------------------------------
freq_ill_class_O

relu
sgd
185
Tasa de acierto: 0.787 +/- 0.018
Tasa de precision 0.782 +/- 0.019
Tasa de exactitud: 0.787 +/- 0.018
Tasa F1-Score 0.783 +/- 0.019
---------------------------------------------------------------
freq_all_class_P

relu
sgd
135
Tasa de acierto: 0.848 +/- 0.013
Tasa de precision 0.848 +/- 0.013
Tasa de exactitud: 0.848 +/- 0.013
Tasa F1-Score 0.847 +/- 0.013
---------------------------------------------------------------
freq_ill_class_P

logistic
sgd
90
Tasa de acierto: 0.819 +/- 0.016
Tasa de precision 0.817 +/- 0.017
Tasa de exactitud: 0.819 +/- 0.016
Tasa F1-Score 0.816 +/- 0.017
---------------------------------------------------------------


In [33]:
resultados_clases(name_class_fc)

fc_all_class_O

relu
sgd
200
Tasa de acierto: 0.849 +/- 0.014
Tasa de precision 0.849 +/- 0.014
Tasa de exactitud: 0.849 +/- 0.014
Tasa F1-Score 0.848 +/- 0.014
---------------------------------------------------------------
fc_ill_class_O

logistic
sgd
85
Tasa de acierto: 0.788 +/- 0.018
Tasa de precision 0.788 +/- 0.019
Tasa de exactitud: 0.788 +/- 0.018
Tasa F1-Score 0.784 +/- 0.019
---------------------------------------------------------------
fc_all_class_P

relu
sgd
125
Tasa de acierto: 0.903 +/- 0.01
Tasa de precision 0.904 +/- 0.01
Tasa de exactitud: 0.903 +/- 0.01
Tasa F1-Score 0.903 +/- 0.01
---------------------------------------------------------------
fc_ill_class_P

logistic
sgd
100
Tasa de acierto: 0.88 +/- 0.015
Tasa de precision 0.879 +/- 0.016
Tasa de exactitud: 0.88 +/- 0.015
Tasa F1-Score 0.879 +/- 0.016
---------------------------------------------------------------


In [34]:
resultados_clases(name_class_rf)

rf_all_class_O

relu
sgd
145
Tasa de acierto: 0.817 +/- 0.014
Tasa de precision 0.816 +/- 0.015
Tasa de exactitud: 0.817 +/- 0.014
Tasa F1-Score 0.815 +/- 0.015
---------------------------------------------------------------
rf_ill_class_O

logistic
sgd
115
Tasa de acierto: 0.752 +/- 0.039
Tasa de precision 0.754 +/- 0.022
Tasa de exactitud: 0.752 +/- 0.039
Tasa F1-Score 0.751 +/- 0.022
---------------------------------------------------------------
rf_all_class_P

relu
sgd
135
Tasa de acierto: 0.874 +/- 0.013
Tasa de precision 0.875 +/- 0.013
Tasa de exactitud: 0.874 +/- 0.013
Tasa F1-Score 0.874 +/- 0.013
---------------------------------------------------------------
rf_ill_class_P

relu
sgd
140
Tasa de acierto: 0.861 +/- 0.016
Tasa de precision 0.862 +/- 0.016
Tasa de exactitud: 0.861 +/- 0.016
Tasa F1-Score 0.861 +/- 0.016
---------------------------------------------------------------


# Mejor configuración

## Multi-clase

In [35]:
names = ['fc_class_p_all', 'fc_class_p_ill']
names_features=['atc', 'cie', 'cie_atc']
features = []
for n in names:
    for f in names_features:
        with open("feature_selection/best/{}_{}.txt".format(n, f), "r") as file:
            features.append(eval(file.readline()))
paths_CLASS = ['/class/P_WC_A_', '/class/P_WC_WO_']
names_CLASS=['fc_all_class_P_atc', 'fc_all_class_P_cie', 'fc_all_class_P_cie_atc',
             'fc_ill_class_P_atc', 'fc_ill_class_P_cie', 'fc_ill_class_P_cie_atc']

In [36]:
k=['all','ill']
for p, i in zip(paths_CLASS, k):
    if i=='all':
        names = names_CLASS[0:4]
        feat = features[0:4]
    else:
        names = names_CLASS[4:8]
        feat = features[4:8]
    for n, f in zip( names, feat):
        for m in param:        
            if path.exists('results/MLP/MLP_hyper_{}_{}.csv'.format(m,n)): 
                print('Ya existe el hyperparametro:', n, m)
            else:
                hyper_MLP(p, f, n, m, True)
                print()
                print('--------------------------------------------------------')
                print()

        if path.exists('results/MLP/MLP_predict_{}.csv'.format(n)): 
            print('Ya existe los resultados:', n)
        else:
            predict_MLP(p, f, n, True)
            print()
            print('--------------------------------------------------------')
            print()

Ya existe el hyperparametro: fc_all_class_P_atc activation
Ya existe el hyperparametro: fc_all_class_P_atc solver
Ya existe el hyperparametro: fc_all_class_P_atc hidden_layer_sizes
Ya existe los resultados: fc_all_class_P_atc
Ya existe el hyperparametro: fc_all_class_P_cie activation
Ya existe el hyperparametro: fc_all_class_P_cie solver
Ya existe el hyperparametro: fc_all_class_P_cie hidden_layer_sizes
Ya existe los resultados: fc_all_class_P_cie
Ya existe el hyperparametro: fc_all_class_P_cie_atc activation
Ya existe el hyperparametro: fc_all_class_P_cie_atc solver
Ya existe el hyperparametro: fc_all_class_P_cie_atc hidden_layer_sizes
Ya existe los resultados: fc_all_class_P_cie_atc
Ya existe el hyperparametro: fc_ill_class_P_atc activation
Ya existe el hyperparametro: fc_ill_class_P_atc solver
Ya existe el hyperparametro: fc_ill_class_P_atc hidden_layer_sizes
Ya existe los resultados: fc_ill_class_P_atc
Ya existe el hyperparametro: fc_ill_class_P_cie activation
Ya existe el hyperpar

In [37]:
resultados_clases(names_CLASS)

fc_all_class_P_atc

relu
sgd
58
Tasa de acierto: 0.742 +/- 0.088
Tasa de precision 0.798 +/- 0.038
Tasa de exactitud: 0.742 +/- 0.088
Tasa F1-Score 0.737 +/- 0.086
---------------------------------------------------------------
fc_all_class_P_cie

tanh
sgd
46
Tasa de acierto: 0.54 +/- 0.094
Tasa de precision 0.644 +/- 0.106
Tasa de exactitud: 0.54 +/- 0.094
Tasa F1-Score 0.548 +/- 0.066
---------------------------------------------------------------
fc_all_class_P_cie_atc

relu
sgd
42
Tasa de acierto: 0.899 +/- 0.012
Tasa de precision 0.9 +/- 0.013
Tasa de exactitud: 0.899 +/- 0.012
Tasa F1-Score 0.898 +/- 0.013
---------------------------------------------------------------
fc_ill_class_P_atc

relu
sgd
36
Tasa de acierto: 0.728 +/- 0.075
Tasa de precision 0.788 +/- 0.046
Tasa de exactitud: 0.728 +/- 0.075
Tasa F1-Score 0.721 +/- 0.077
---------------------------------------------------------------
fc_ill_class_P_cie

relu
sgd
52
Tasa de acierto: 0.328 +/- 0.112
Tasa de precision 0.597

## Multi-label

In [38]:
names = ['fc_label_p_all', 'fc_label_p_ill']
names_features=['atc', 'cie', 'cie_atc']
features = []
for n in names:
    for f in names_features:
        with open("feature_selection/best/{}_{}.txt".format(n, f), "r") as file:
            features.append(eval(file.readline()))
    features += [['Edad', 'Sexo']]
paths_label = ['/label/P_WL_A_', '/label/P_WL_WO_']
names_label=['fc_all_label_P_atc', 'fc_all_label_P_cie', 'fc_all_label_P_cie_atc', 'fc_all_label_P_E_S', 
             'fc_ill_label_P_atc', 'fc_ill_label_P_cie', 'fc_ill_label_P_cie_atc', 'fc_ill_label_P_E_S']

In [39]:
k=['all','ill']
for p, i in zip(paths_label, k):
    if i=='all':
        names = names_label[0:4]
        feat = features[0:4]
    else:
        names = names_label[4:8]
        feat = features[4:8]
    for n, f in zip( names, feat):
        for m in param:        
            if path.exists('results/MLP/MLP_hyper_{}_{}.csv'.format(m,n)): 
                print('Ya existe el hyperparametro:', n, m)
            else:
                hyper_MLP(p, f, n, m, True)
                print()
                print('--------------------------------------------------------')
                print()

        if path.exists('results/MLP/MLP_predict_{}.csv'.format(n)): 
            print('Ya existe los resultados:', n)
        else:
            predict_MLP(p, f, n, True)
            print()
            print('--------------------------------------------------------')
            print()

Ya existe el hyperparametro: fc_all_label_P_atc activation
Ya existe el hyperparametro: fc_all_label_P_atc solver
Ya existe el hyperparametro: fc_all_label_P_atc hidden_layer_sizes
Ya existe los resultados: fc_all_label_P_atc
Ya existe el hyperparametro: fc_all_label_P_cie activation
Ya existe el hyperparametro: fc_all_label_P_cie solver
Ya existe el hyperparametro: fc_all_label_P_cie hidden_layer_sizes
Ya existe los resultados: fc_all_label_P_cie
Ya existe el hyperparametro: fc_all_label_P_cie_atc activation
Ya existe el hyperparametro: fc_all_label_P_cie_atc solver
Ya existe el hyperparametro: fc_all_label_P_cie_atc hidden_layer_sizes
Ya existe los resultados: fc_all_label_P_cie_atc
Ya existe el hyperparametro: fc_all_label_P_E_S activation
Ya existe el hyperparametro: fc_all_label_P_E_S solver
Ya existe el hyperparametro: fc_all_label_P_E_S hidden_layer_sizes
Ya existe los resultados: fc_all_label_P_E_S
Ya existe el hyperparametro: fc_ill_label_P_atc activation
Ya existe el hyperpar

In [40]:
resultados_etiquetas(names_label)

fc_all_label_P_atc

relu
sgd
52
Tasa de acierto: 0.82 +/- 0.035
Tasa de Hamming Loss: 0.067 +/- 0.013
Tasa de precision(macro) 0.937 +/- 0.02
Tasa de precision(micro) 0.946 +/- 0.027
Tasa de exactitud(macro): 0.885 +/- 0.012
Tasa de exactitud(micro): 0.91 +/- 0.011
Tasa F1-Score(macro) 0.909 +/- 0.011
Tasa F1-Score(micro) 0.927 +/- 0.012
---------------------------------------------------------------
fc_all_label_P_cie

relu
sgd
48
Tasa de acierto: 0.455 +/- 0.196
Tasa de Hamming Loss: 0.23 +/- 0.094
Tasa de precision(macro) 0.724 +/- 0.125
Tasa de precision(micro) 0.726 +/- 0.136
Tasa de exactitud(macro): 0.853 +/- 0.057
Tasa de exactitud(micro): 0.906 +/- 0.056
Tasa F1-Score(macro) 0.764 +/- 0.056
Tasa F1-Score(micro) 0.794 +/- 0.061
---------------------------------------------------------------
fc_all_label_P_cie_atc

relu
sgd
44
Tasa de acierto: 0.895 +/- 0.012
Tasa de Hamming Loss: 0.037 +/- 0.004
Tasa de precision(macro) 0.953 +/- 0.012
Tasa de precision(micro) 0.967 +/- 0.007
T